In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
def BN(mutated_genes):
    
    # Inputs 
    genes = {
        "Shh" : True,
        "Wnt" : True,
        "GF" : False, # EGF or PDGF or IGF1,
        "IL17" : True, # Cytokine,
        "Cytokine" : True,
        "TNF" : True,
        "PSEN" : False,
        "TGFb" : True,
        "S1P" : False,
        "Antigen" : False,
        # G-coupled Proteins,
        "Dopamine" : False,
        "GABA" : False,
        "Ach" : False,
        "HT" : False,
        "PGE2" : False, #,
        "EDN1" : False,
        "Norepinephrine" : False,
        "F2" : False,
        "Estrogen" : False,
        "Testosterone" : False,
        "Progesterone" : False, #,
        "NF1" : True
    }
    
    

    ###############Giving value to the nodes (genes) based on Kegg information and assigning truth value to the selection of nodes
    # G-coupled Proteins
    genes["DRD2"] = genes["Dopamine"] 

    genes["HTR3A"] = genes["HT"] 
    genes["HTR4"] = genes["HT"]
    genes["ACHE"] = genes["Ach"]
    genes["GABRA1"] = genes["GABA"] 
    genes["EDNRA"] = genes["EDN1"]
    genes["ADRB1"] = genes["Norepinephrine"]
    genes["ADRB2"] = genes["Norepinephrine"]
    genes["FKBP1A"] = 0
    genes["CA"] = 0
    genes["THBD"] = genes["F2"]
    genes["GP1BA"] = 0

#####group of nodes/ assign from Kegg/ do not get the fault values
    genes["GiGPCR"] = genes["GABRA1"] or genes["ACHE"] or genes["DRD2"] or genes["EDNRA"] or genes["ADRB2"] or genes["F2"]
    genes["GsGPCR"] = genes["HTR4"] or genes["PGE2"] or genes["ADRB1"]    
    
    # Histone DeAcetylation
    genes["HDAC"] = False
    
    # Wnt Pathway
    genes["Frizzled"] = genes["Wnt"]
    genes["DV1"] = genes["Frizzled"]
    genes["GSK3b"] = genes["IL17"] and not (genes["DV1"] or genes["GF"])
    genes["bCatenin"] = not genes["GSK3b"]
    # Notch 
    genes["Notch"] = (genes["PSEN"] and not genes["DV1"])
    
     # Receptors 
    genes["KDR"] = genes["GF"] or (genes["Notch"] and not genes["HDAC"])
    genes["FLT1"] = genes["GF"] 
    genes["FLT3"] = genes["GF"]
    genes["FLT4"] = genes["GF"]
    genes["RET"] = genes["GF"]
    genes["FGF2"] = genes["GF"]
    genes["EGFR"] = genes["GF"]
    genes["PDGFR"] = genes["GF"]
    genes["KIT"] = genes["GF"]
    genes["ERBB2"] = genes["GF"] or (genes["Notch"] and not genes["HDAC"])
    genes["IGF1R"] = genes["GF"] 
    genes["ALK"] = 0 
    genes["Syk"] = genes["Antigen"]
    genes["BTK"] = genes["Syk"] or genes["GP1BA"]        
    genes["RTK"] = (genes["KDR"] or genes["FLT3"] or genes["FLT4"] or genes["EGFR"] or genes["FGF2"] or genes["PDGFR"] or genes["KIT"] or genes["ERBB2"] or genes["IGF1R"])
    
     # Androgen Signaling
    genes["AR"] = genes["Testosterone"] or genes["Progesterone"]
    # cAMP-PKA Pathway 
    genes["Gs"] = genes["GsGPCR"] and not genes["CA"]
    genes["Gi"] = genes["GiGPCR"] or genes["S1P"]
    genes["AC"] = genes["Gs"] and not genes["Gi"]
    genes["cAMP"] = genes["AC"] and not (genes["AR"] or genes["IGF1R"])
    genes["PKA"] = genes["cAMP"]
    
    # Actin
    genes["ROCK"] = (genes["TGFb"] or genes["F2"] or genes["S1P"]) and not genes["PKA"]



     # Calcium Signaling
    genes["Gq"] = genes["GiGPCR"] or genes["S1P"]
    genes["PLCb"] = genes["Gq"] 
    genes["PLCg"] = genes["RTK"] or genes["ALK"] or genes["ERBB2"] or genes["BTK"]
    genes["IP3R"] = genes["PLCg"] or genes["PLCb"] 

    genes["Ca2p"] = genes["IP3R"]
    genes["DAG"] = genes["PLCg"] or genes["PLCb"]
    genes["PKC"] = (genes["DAG"] and genes["Ca2p"])
    
    # MAPK-ERK Pathway
    genes["JAK"] = (genes["Cytokine"] or genes["RTK"] or genes["ALK"])
    genes["Shc"] = genes["RTK"]
    genes["Grb2"] = genes["Shc"]
    genes["Sos"] = genes["Grb2"]
    genes["Ras"] = (genes["Sos"] or genes["cAMP"] or genes["RET"] or genes["ALK"]) and not genes["NF1"]
    genes["Raf"] = (genes["Ras"] or genes["PKC"]) and not genes["PKA"]
    genes["MEK"] = genes["Raf"]
    genes["ERK"] = (genes["MEK"] or genes["IL17"])

    # Estrogen Signaling
    genes["ESR1"] = genes["ERK"] or genes["Estrogen"]
    genes["SRC"] = genes["ESR1"] or genes["KDR"] or genes["ERBB2"] or genes["EGFR"]

    # PI3K-Akt Pathway
    genes["PTEN"] = genes["ROCK"]
    genes["PI3K"] = genes["RTK"] or genes["Ras"] or genes["JAK"] or genes["GiGPCR"] or genes["GP1BA"] or genes["ALK"] or genes["IGF1R"] or genes["Syk"] or genes["SRC"]
    genes["PIP3"] = genes["PI3K"] and not genes["PTEN"]
    genes["PDK1"] = genes["PIP3"]
    genes["AKT"] = genes["PDK1"]
    
     # TNF Pathway
    genes["TNFR"] = genes["TNF"]
    genes["TRAF2"] = genes["TNFR"] or genes["IL17"]
    genes["RIP1"] = genes["TNFR"]

    # NFKB Pathway
    genes["IKK"] = (genes["TRAF2"] or genes["RIP1"] or genes["PDK1"] or genes["BTK"])
    genes["NFKB"] = genes["IKK"] or genes["ROCK"]

    # AMP Pathway
    genes["LKB1"] = genes["cAMP"]
    genes["AMPK"] = genes["LKB1"]

    # mTOR Pathway
    genes["RSK2"] = genes["ERK"]
    genes["TSC"] = ((genes["GSK3b"] and not genes["AKT"]) or genes["AMPK"]) and not (genes["IKK"] or genes["RSK2"] or genes["ERK"] or genes["AKT"])
    genes["Rheb"] = not genes["TSC"]
    genes["mTORc1"] = (genes["AKT"] or genes["Rheb"] or genes["IKK"]) and not genes["AMPK"]
    genes["S6K"] = genes["mTORc1"] or genes["ERK"] or genes["TGFb"]
    genes["mTORc2"] = (genes["PI3K"] or genes["IKK"]) and not genes["S6K"]
    genes["STAT3"] = genes["JAK"]
    genes["VEGF"] = genes["STAT3"] or genes["KDR"]
    
    
    
    # DNA Damage and Methylation
    genes["eNOS"] = genes["VEGF"] or genes["AKT"] or genes["ADRB1"] or genes["ADRB2"]
    genes["NO"] = genes["eNOS"]
    genes["OG"] = genes["NO"] 
    genes["MGMT"] = genes["OG"] and genes["HDAC"] 
    genes["DNADamage"] = genes["OG"] and not (genes["MGMT"] or genes["HDAC"]) 
    
    
    # Hypoxia
    genes["IDH"] = 0
    genes["aKG"] = genes["IDH"]
    genes["HIF1a"] = genes["NFKB"] or genes["STAT3"] or genes["ERK"] or genes["S6K"]
    genes["Hypoxia"] = genes["HIF1a"] or (genes["NO"] and not (genes["aKG"] or genes["PKC"])) 
    genes["iNOS"] = genes["Hypoxia"]
    genes["COX2"] = (genes["NFKB"] or (genes["VEGF"] and genes["Ca2p"]) or genes["PGE2"])

    # ER Stress
    genes["ERStress"] = genes["IP3R"] or genes["Hypoxia"] 
    genes["PERK"] = genes["ERStress"]
    genes["ELF2a"] = genes["PERK"]
    genes["IRE1a"] = genes["ERStress"] 
    
    
    # TGFb Pathway
    genes["TGFbR"] = genes["TGFb"] or genes["HIF1a"]
    genes["SMAD23"] = genes["TGFbR"] and not (genes["TNF"] or genes["Cytokine"] or genes["FKBP1A"])

    # JNK Pathway
    genes["MEKK1"] = genes["TRAF2"] or genes["TGFbR"] or genes["SRC"]
    genes["JNK"] = (genes["Ras"] and not genes["AKT"]) or genes["MEKK1"] or genes["IRE1a"]

    # cGMP-PKG Pathway
    genes["sGC"] = genes["NO"] or genes["eNOS"]
    genes["cGMP"] = genes["sGC"] 
    genes["PKG"] = genes["cGMP"]

    # PPAR signaling
    genes["PPARa"] = genes["PKA"]
    genes["PPARg"] = genes["mTORc1"]
    
    
    # Targets
    genes["PIAS"] = genes["DNADamage"]
    genes["cMyc"] = (genes["bCatenin"] or genes["ESR1"] or (genes["STAT3"] and not genes["PIAS"]))
    genes["CREB"] = genes["PKA"] or genes["PKG"] or genes["RSK2"] or genes["AKT"] or genes["ELF2a"]
    genes["CHOP"] = genes["CREB"]
    genes["FOXO"] = (genes["STAT3"] or genes["AMPK"] or genes["JNK"]) and not (genes["AKT"] or genes["ERK"] or (genes["IKK"] and genes["PDK1"]) or genes["mTORc2"])
    genes["p15"] = (genes["FOXO"] or genes["SMAD23"]) and not (genes["HDAC"] or genes["cMyc"])


    # Hedgehog Pathway
    genes["GliR"] = genes["PKA"] and genes["GSK3b"]
    genes["GliA"] = genes["Shh"]
    genes["Gli"] = genes["GliA"] and not genes["GliR"] 
    
    
    # p53 Signaling
    genes["fos"] = genes["JNK"] or (genes["CREB"] and genes["PKA"]) or (genes["cMyc"] and genes["ERK"])
    genes["jun"] = genes["bCatenin"] or genes["JNK"] or (genes["CREB"] and genes["PKA"])
    genes["ATM"] = genes["DNADamage"] or genes["FOXO"]
    genes["CHK1"] = genes["ATM"]
    genes["p16INK4a"] = (genes["SMAD23"] and not genes["cMyc"])
    genes["p14ARF"] = (genes["DNADamage"])
    genes["MDM2"] = genes["AKT"] and not genes["p14ARF"]
    genes["p53"] = ((genes["JNK"] or genes["CHK1"] or genes["ERStress"] or genes["NO"] or genes["Hypoxia"]) and not genes["MDM2"])
    genes["PCNA"] = genes["p53"]
    genes["CDC25"] = not genes["CHK1"]
    genes["p21"] = ((genes["p53"] or genes["FOXO"]) or (genes["STAT3"] and not genes["PIAS"])) and not genes["AKT"]

     # Apoptosis
    genes["MMP1"] = genes["PPARg"]
    genes["MMP2"] = genes["SRC"] or genes["GABRA1"] or genes["PKC"] or genes["NFKB"] or (genes["fos"] and genes["jun"])
    genes["PAI"] = (genes["p53"] or genes["SMAD23"] or genes["HIF1a"]) and not genes["THBD"]
    genes["PARP1"] = genes["DNADamage"]
    genes["Calpain"] = (genes["ERStress"] or genes["PARP1"])
    genes["TERT"] = 0

    genes["Rac"] = (genes["PIP3"] and genes["VEGF"])
    genes["cPLA2"] = (genes["Ca2p"] and genes["VEGF"])

    genes["CCND1"] = (genes["bCatenin"] or genes["Gli"] or genes["ESR1"] or (genes["fos"] and genes["jun"])) and not (genes["p21"] or genes["p15"] or (genes["GSK3b"] and not genes["AKT"]) or genes["PCNA"])
    genes["CDK4"] = (genes["CDC25"] or (genes["fos"] and genes["jun"])) and not genes["p16INK4a"]

    genes["CASP12"] = genes["Calpain"]
    genes["cFLIP"] = genes["NFKB"] and not genes["JNK"]
    genes["FasL"] = genes["FOXO"] or genes["fos"] or genes["jun"] or genes["TNF"]
    genes["FADD"] = (genes["FasL"] and not genes["COX2"])
    genes["CASP8"] = (genes["FADD"] or genes["TNFR"]) and not genes["cFLIP"]
    genes["Bid"] = genes["p53"] or genes["JNK"] or genes["CASP8"]
    genes["Mcl1"] = (genes["CREB"] and genes["AKT"]) or (genes["STAT3"]) 
    genes["puma"] = genes["p53"]
    genes["noxa"] = genes["p53"]
    genes["XIAP"] = genes["TRAF2"]
    
    genes["Bad"] = (genes["PKG"] or genes["DNADamage"]) and not (genes["AKT"] or genes["PKA"])
    genes["BCL2"] = (genes["Gli"] or genes["ESR1"] or (genes["CREB"] and genes["AKT"]) or (genes["STAT3"] and not genes["PIAS"])) and not (genes["puma"] or genes["noxa"] or genes["Bid"] or genes["CHOP"] or genes["PPARa"] or genes["JNK"] or genes["Bad"])
    genes["BCLxl"] = (genes["STAT3"] and not genes["PIAS"]) or (genes["NFKB"])
    genes["Bim"] = (genes["FOXO"] or genes["fos"] or genes["jun"] or genes["DNADamage"]) and not genes["Mcl1"]
    genes["Bak"] = (genes["p53"] or genes["Bim"]) and not (genes["BCL2"] or genes["BCLxl"])
    genes["Bax"] = (genes["p53"] or genes["Bid"]) and not (genes["BCL2"] or genes["BCLxl"])
    
    
    # overriding the mutated genes over the rules 
    # of the network defined as above
    for gene in mutated_genes:
        genes[gene] = mutated_genes[gene]


        
    AVGBak = np.average([genes["Bak"],genes["p53"],genes["Bim"],not genes["BCL2"],not genes["BCLxl"]],weights=[1/6,1/6,1/6,1/4,1/4])
    AVGBax = np.average([genes["Bax"],genes["p53"],genes["Bid"],not genes["BCL2"],not genes["BCLxl"]],weights=[1/6,1/6,1/6,1/4,1/4])
    AVGBid = np.average([genes["Bid"],genes["p53"],genes["JNK"],genes["CASP8"]],weights=[1/4,1/4,1/4,1/4])
    AVGCASP8 = np.average([genes["CASP8"],genes["FADD"],genes["TNFR"],not genes["cFLIP"]],weights=[1/6,1/6,1/6,1/2])
    AVGDNADamage = np.average([genes["DNADamage"],genes["OG"],not genes["MGMT"],not genes["HDAC"]],weights=[1/4,1/4,1/4,1/4])
    AVGBCL2 = np.average([genes["BCL2"],genes["Gli"],genes["ESR1"],genes["CREB"],genes["AKT"],genes["STAT3"],not genes["PIAS"],not genes["puma"],not genes["noxa"], not genes["Bid"], not genes["CHOP"],not genes["PPARa"],not genes["JNK"],not genes["Bad"]],weights=[1/12,1/12,1/12,1/12,1/12,1/12,1/16,1/16,1/16,1/16,1/16,1/16,1/16,1/16])
    AVGBCLxl = np.average([genes["BCLxl"],genes["STAT3"],genes["NFKB"],not genes["PIAS"]],weights=[1/6,1/6,1/6,1/2])
    AVGMcl1 = np.average([genes["CREB"],genes["AKT"],genes["STAT3"]],weights=[1/3,1/3,1/3])
    AVGcFLIP = np.average([genes["NFKB"],not genes["JNK"]],weights=[1/2,1/2])
    AVGCDK4 = np.average([genes["CDK4"],genes["CDC25"],genes["fos"],genes["jun"],not genes["p16INK4a"]],weights=[1/8,1/8,1/8,1/8,1/2])
    AVGCCND1 = np.average([genes["CCND1"],genes["bCatenin"],genes["Gli"],genes["ESR1"],genes["fos"],genes["jun"],genes["AKT"],not genes["p21"],not genes["p15"],not genes["GSK3b"],not genes["PCNA"]],weights=[1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/4,1/4,1/4,1/4])
    AVGAR = np.average([genes["Testosterone"],genes["Progesterone"]],weights=[1/2,1/2])
    
    
     #### based on whether or not RB protein is expressed 
    proArrest = np.average([genes["DNADamage"],genes["CHK1"]])
    antiArrest = np.average([genes["HDAC"],genes["CDK4"],genes["CCND1"],genes["AR"]])
    
    ###based on whether or not CASP3 is Cleaved
    proApoptosis = np.average([genes["Bak"], genes["Bax"], genes["Bid"], genes["noxa"], genes["puma"], genes["CASP12"], genes["CASP8"], genes["DNADamage"]]) 
    antiApoptosis = np.average([genes["TERT"], genes["XIAP"], genes["BCL2"], genes["BCLxl"], genes["Mcl1"], genes["cFLIP"]])
    
    ####Mixture of factors , refers to Kegg pathways in cancer
    proAngiogenesis = np.average([genes["iNOS"], genes["COX2"], genes["VEGF"], genes["fos"], genes["jun"], genes["Hypoxia"], genes["MMP1"], genes["MMP2"], genes["FLT1"], genes["Rac"], genes["cPLA2"], genes["ERBB2"], genes["KDR"], genes["EDNRA"]])
    antiAngiogenesis = np.average([genes["p53"], genes["PAI"]])
    

    
    if antiArrest == 0:
        antiArrest = 0.01
    if antiApoptosis == 0:
        antiApoptosis = 0.01
    if proAngiogenesis == 0:
        proAngiogenesis = 0.01
    
    return proApoptosis,antiApoptosis,proArrest,antiArrest,proAngiogenesis,antiAngiogenesis

In [3]:
Label =['DRD2', 'HTR3A', 'HTR4', 'ACHE', 'GABRA1', 'EDNRA', 'ADRB1', 'ADRB2', 'FKBP1A', 'CA', 'THBD', 'GP1BA', 'HDAC', 'Frizzled', 'DV1', 'GSK3b', 'bCatenin', 'Notch', 'KDR', 'FLT1', 'FLT3', 'FLT4', 'RET', 'FGF2', 'EGFR', 'PDGFR', 'KIT', 'ERBB2', 'IGF1R', 'ALK', 'Syk', 'BTK', 'AR', 'Gs', 'Gi', 'AC', 'cAMP', 'PKA', 'ROCK', 'Gq', 'PLCb', 'PLCg', 'IP3R', 'DAG', 'PKC', 'JAK', 'Shc', 'Grb2', 'Sos', 'Ras', 'Raf', 'MEK', 'ERK', 'ESR1', 'SRC', 'PTEN', 'PI3K', 'PIP3', 'PDK1', 'AKT', 'TNFR', 'TRAF2', 'RIP1', 'IKK', 'NFKB', 'LKB1', 'AMPK', 'RSK2', 'TSC', 'Rheb', 'mTORc1', 'S6K', 'mTORc2', 'STAT3', 'VEGF', 'eNOS', 'OG', 'MGMT', 'IDH1', 'HIF1a', 'iNOS', 'COX2', 'PERK', 'ELF2a', 'IRE1a', 'TGFbR', 'SMAD23', 'MEKK1', 'JNK', 'sGC', 'PKG', 'PPARa', 'PPARg', 'PIAS', 'cMyc', 'CREB', 'CHOP', 'FOXO', 'p15', 'GliR', 'GliA', 'fos', 'jun', 'ATM', 'CHK1', 'p16INK4a', 'p14ARF', 'MDM2', 'p53', 'PCNA', 'CDC25', 'p21', 'MMP1', 'MMP2', 'PAI', 'PARP1', 'Calpain', 'TERT', 'Rac', 'cPLA2', 'CCND1', 'CDK4', 'CASP12', 'cFLIP', 'FasL', 'FADD', 'CASP8', 'Bid', 'Mcl1', 'puma', 'noxa', 'XIAP', 'Bad', 'BCL2', 'BCLxl', 'Bim', 'Bak', 'Bax','NF1'] # ,'Shh','Wnt','GF','IL17','Cytokine','TNF','PSEN','TGFb','S1P','Antigen','Dopamine','GABA','Ach','HT','PGE2','EDN1','Norepinephrine','F2','Estrogen','Testosterone','Progesterone','NF1'
N = len(Label)
col = ['proApoptosis','antiApoptosis','proArrest','antiArrest','proAngiogenesis','antiAngiogenesis','ConvexSum','ApopRatio','ArrestRatio','AngioRatio']

In [4]:
len(Label)

139

In [5]:
def Fault_to_Metric(mutated_genes):
    proApoptosis,antiApoptosis,proArrest,antiArrest,proAngiogenesis,antiAngiogenesis = BN(mutated_genes)
    CD = np.true_divide(proApoptosis,antiApoptosis) #CD is apoptosis
    ARREST = np.true_divide(proArrest,antiArrest)
    ANGIO = np.true_divide(antiAngiogenesis,proAngiogenesis)
    ConvexSum = np.average([CD,ARREST],weights=[14.0/20,6.0/20])
    metric = pd.Series({'proApoptosis':proApoptosis,'antiApoptosis':antiApoptosis,'proArrest':proArrest,'antiArrest':antiArrest,'proAngiogenesis':proAngiogenesis,'antiAngiogenesis':antiAngiogenesis,'ConvexSum': ConvexSum,'ApopRatio':CD,'ArrestRatio':ARREST,'AngioRatio':ANGIO})
    
    return metric

In [6]:
# GDSC keywords for mutation in GBM
# This parts will be changed based on different cell lines

# cnaGBM2 = [p16INK4a : False, p14ARF : False]
# cnaGBM12 = [PDGFR : True]
# cnaGBM123  = [EGFR : True]
# cnaGBM26 = [MDM2 : True]
# cnaGBM25 = [CDK4 : True]
# cnaGBM68 = [Fas : False]

cell_lines = ['NONE','42-MG-BA','A172','AM-38','CCF-STTG1','LN-229','T98G','U-87-MG','YKG-1']



mutations = [
    { #None
        "DRD2":False
    }, { #'42-MG-BA'
        "p16INK4a":False,
        "p14ARF":False,
        "PDGFR":True,
        "PTEN":False,
        "p53":False
    }, { #'A172'
        "p16INK4a":False,
        "p14ARF":False,
        "EGFR":True,
        "p53":False
    }, { #'AM-38'
        "p16INK4a":False,
        "p14ARF":False,
        "Raf":True
    }, { #'CCF-STTG1'
        "EGFR":True,
        "MDM2":True,
        "PTEN":False
    } , { #'LN-229'
        "FasL":False,
        "p16INK4a":False,
        "p14ARF":False,
        "EGFR":True,
        "Sos":True
    }, { #'T98G'
        "FasL":False,
        "p16INK4a":False,
        "p14ARF":False,
        "p53":False
    }, { #'U-87-MG' NF1
        "FasL":False,
        "p16INK4a":False,
        "p14ARF":False,
        "PTEN":False,
        "NF1":False
    }, { #'YKG-1' NF1
        "p16INK4a":False,
        "p14ARF":False,
        "p53":False,
        "PTEN":False,
        "PI3K":True,
        "NF1":False
    }]



df = pd.DataFrame(columns=col,index=cell_lines)

for i in range(len(cell_lines)):
    
    m = Fault_to_Metric(mutations[i]) #passing those mutation into the BN
 
    df.loc[cell_lines[i]] = m
    
df['mutations'] = mutations

df.to_csv('CELLLINES_FAULTS_GBM.csv')



print(df['ConvexSum'])

NONE         2.25
42-MG-BA     2.25
A172         2.25
AM-38        2.25
CCF-STTG1    2.25
LN-229       2.25
T98G         2.25
U-87-MG      2.25
YKG-1        2.25
Name: ConvexSum, dtype: object


In [7]:
def Drug_to_Metric(mutations,actions):
    # joining both mutations and actions
    # works for Python >=3.5
    all_mutations = mutations.copy()   # start with "mutations"
    all_mutations.update(actions)    # modify "actions"
    
    proApoptosis,antiApoptosis,proArrest,antiArrest,proAngiogenesis,antiAngiogenesis = BN(all_mutations)
    CD = np.true_divide(proApoptosis,antiApoptosis) #CD is mutated_nodes
    ARREST = np.true_divide(proArrest,antiArrest)
    ANGIO = np.true_divide(antiAngiogenesis,proAngiogenesis)
    ConvexSum = np.average([CD,ARREST],weights=[14.0/20,6.0/20])
    metric = pd.Series({'proApoptosis':proApoptosis,'antiApoptosis':antiApoptosis,'proArrest':proArrest,'antiArrest':antiArrest,'proAngiogenesis':proAngiogenesis,'antiAngiogenesis':antiAngiogenesis,'ConvexSum': ConvexSum,'ApopRatio':CD,'ArrestRatio':ARREST,'AngioRatio':ANGIO})
    
    return metric

In [8]:
DFCOMB = pd.read_excel('KEY.xlsx',index_col=0, sheet_name = 'COMBO')

Drugs = DFCOMB['Name']
all_actions = []



for i in range(len(Drugs)):
    actions={}
    action_values=str(DFCOMB['truth_values'].iloc[i]).split(',')
    

    for j,gene in enumerate(DFCOMB['nodes_gene'].iloc[i].split(',')):
        gene=gene.strip()  # to remove white space
     
        if action_values[j]=='True':
            actions[gene]=True
        else:
            actions[gene]=False
            
    all_actions.append(actions)
            
    

In [9]:
CDV = pd.DataFrame(columns=cell_lines,index=Drugs)
                
for i in range(len(Drugs)):               
    for Select in range(len(cell_lines)):
        THISmutations = mutations[Select]
        
        
        metric = Drug_to_Metric(THISmutations,all_actions[i])
        CDV.loc[Drugs[i]][Select] = metric[6] 

X = CDV['NONE']
Y = CDV.drop(['NONE'],axis=1)

for i in X.index:
    Y.loc[i] = np.true_divide(Y.loc[i],X.loc[i]) 

Y = Y.T
X = Y['NODRUG']

for i in X.index:
    Y.loc[i] = np.true_divide(Y.loc[i],X.loc[i]) 
    
Y = Y.T

CDV = Y
CDV = CDV.T
CDV.to_csv('DRUGS_COMBO_GBM.csv')

In [10]:
INDEX = []

P_ACTIONS = []

D = len(Drugs)
Counts = itertools.combinations_with_replacement(range(D),2) 
for c in Counts:
    INDEX.append(c)
    
for c in range(len(INDEX)): 
    Index = INDEX[c]
    Drug1 = Index[0]
    Drug2 = Index[1]
    
    
    actions={}
    for Drug in [Drug1, Drug2]:
        
        action_values=str(DFCOMB['truth_values'].iloc[Drug]).split(',')
    
        for j,gene in enumerate(DFCOMB['nodes_gene'].iloc[Drug].split(',')):
            gene=gene.strip()  # to remove white space
     
            if action_values[j]=='True':
                actions[gene]=True
            else:
                actions[gene]=False
            
        P_ACTIONS.append(actions)

In [11]:
RANGEindex = pd.RangeIndex(len(INDEX))
CDVP = pd.DataFrame(index=RANGEindex,columns=cell_lines)

for c in range(len(INDEX)): 
    Index = INDEX[c]
    Drug1 = Index[0]
    Drug2 = Index[1]
    for Select in range(len(cell_lines)):
        THISmutations = mutations[Select]
       
        
        metric = Drug_to_Metric(THISmutations,P_ACTIONS[c])
        CDVP.loc[c][Select] = metric[6] 

X = CDVP['NONE']
Y = CDVP.drop(['NONE'],axis=1)

for i in X.index:
    Y.loc[i] = np.true_divide(Y.loc[i],X.loc[i]) 
    
Y = Y.T
X = Y[0]

for i in X.index:
    Y.loc[i] = np.true_divide(Y.loc[i],X.loc[i]) 
    
Y = Y.T

CDVP = Y
CDVP.index = INDEX 
CDVP = CDVP.T
CDVP.to_csv('DRUGS_COMBO_PAIRS_GBM.csv')